In [56]:
import pandas as pd
from datetime import date, timedelta
import urllib
import numpy as np
import altair as alt

In [57]:
# NHS File Format
nhs_file_format = "https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/DIRECTORY/COVID-19-total-announced-deaths-DATE.xlsx"
nhs_file_date_format = "%-d-%B-%Y"
nhs_file_directory_format = "%Y/%m"

In [58]:
# Create a list of dates
start_date = date(2020, 4, 1)   # start date
end_date = date.today()
delta = end_date - start_date       # as timedelta
file_dates = [] # initialise empty list

for i in range(delta.days + 1):
    file_dates.append(start_date + timedelta(days=i))

In [59]:
# Create a list URLs
paths = []
real_paths = []

for day in file_dates:
    path = nhs_file_format.replace("DATE",day.strftime(nhs_file_date_format))
    path = path.replace("DIRECTORY",day.strftime(nhs_file_directory_format))
    
    paths.append(path)

for path in paths:
    try:
      urllib.request.urlopen(path)
      real_paths.append(path)
        
    except urllib.error.HTTPError as e:
        pass

In [63]:
real_paths

['https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-4-April-2020.xlsx',
 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-5-April-2020.xlsx',
 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-6-April-2020.xlsx',
 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-7-April-2020.xlsx',
 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-8-April-2020.xlsx',
 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-9-April-2020.xlsx',
 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-10-April-2020.xlsx',
 'https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/COVID-19-total-announced-deaths-11-